In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Preprocess ADMISSIONS Table

In [ ]:
# Load admissions data
df_adm = pd.read_csv("/content/drive/MyDrive/w266: NLP Project/MIMIC_III_data/ADMISSIONS.csv")

# df_adm.head()

In [ ]:
# Convert Strings to Dates. When converting dates, it is safer to use a datetime format. Setting the errors = 'coerce' flag allows for 
#   missing dates but it sets it to NaT (not a datetime) when the string doesn't match the format.
df_adm.ADMITTIME = pd.to_datetime(df_adm.ADMITTIME, format='%Y-%m-%d %H:%M:%S', errors='coerce')
df_adm.DISCHTIME = pd.to_datetime(df_adm.DISCHTIME, format='%Y-%m-%d %H:%M:%S', errors='coerce')
df_adm.DEATHTIME = pd.to_datetime(df_adm.DEATHTIME, format='%Y-%m-%d %H:%M:%S', errors='coerce')

Get the next unplanned admission date for each patient (if it exists). First I'll verify that the dates are in order. Then I'll use the shift() function to get the next admission date.

In [ ]:
df_adm = df_adm.sort_values(['SUBJECT_ID', 'ADMITTIME'])
df_adm = df_adm.reset_index(drop=True)
df_adm['NEXT_ADMITTIME'] = df_adm.groupby('SUBJECT_ID').ADMITTIME.shift(-1)
df_adm['NEXT_ADMISSION_TYPE'] = df_adm.groupby('SUBJECT_ID').ADMISSION_TYPE.shift(-1)

Since I want to predict unplanned re-admissions I will drop (filter out) any future admissions that are ELECTIVE so that only EMERGENCY re-admissions are measured. For rows with 'elective' admissions, replace it with NaT and NaN

In [ ]:
rows = df_adm.NEXT_ADMISSION_TYPE == 'ELECTIVE'
df_adm.loc[rows,'NEXT_ADMITTIME'] = pd.NaT
df_adm.loc[rows,'NEXT_ADMISSION_TYPE'] = np.NaN

In [ ]:
#It's safer to sort right before the fill incase something I did above changed the order
df_adm = df_adm.sort_values(['SUBJECT_ID','ADMITTIME'])

Backfill in the values that I removed. So copy the ADMITTIME from the last emergency and paste it in the NEXT_ADMITTIME for the previous emergency. So I am effectively ignoring/skipping the ELECTIVE admission row completely. Doing this will allow me to calculate the days until the next admission.

In [ ]:
# Back fill. This will take a little while.
df_adm[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']] = df_adm.groupby(['SUBJECT_ID'])[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']].fillna(method = 'bfill')

# Calculate days until next admission
df_adm['DAYS_NEXT_ADMIT'] = (df_adm.NEXT_ADMITTIME - df_adm.DISCHTIME).dt.total_seconds()/(24*60*60)

**Remove NEWBORN admissions and death events**

According to the MIMIC site "Newborn indicates that the HADM_ID pertains to the patient's birth."

I will remove all NEWBORN admission types because in this project I'm not interested in studying births — my primary interest is EMERGENCY and URGENT admissions. I will remove all admissions that have a DEATHTIME because in this project I'm studying re-admissions, not mortality. And a patient who died cannot be re-admitted.

In [ ]:
df_adm = df_adm.loc[df_adm.ADMISSION_TYPE != 'NEWBORN']
df_adm = df_adm.loc[df_adm.DEATHTIME.isnull()]

Make Output Label

For this problem, we are going to classify if a patient will be admitted in the next 30 days. Therefore, we need to create a variable with the output label (1 = readmitted, 0 = not readmitted).

In [ ]:
df_adm['OUTPUT_LABEL'] = (df_adm.DAYS_NEXT_ADMIT < 30).astype('int')
df_adm['DURATION'] = (df_adm['DISCHTIME']-df_adm['ADMITTIME']).dt.total_seconds()/(24*60*60)

#Preprocess NOTEEVENTS Table

# New Section

In [ ]:
# Load data - The noteevents file is huge, so this will take a 1-2min if you load that .csv
df_notes = pd.read_csv("/content/drive/MyDrive/w266: NLP Project/MIMIC_III_data/NOTEEVENTS.csv")

df_notes.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [ ]:
print(df_notes.iloc[10]['TEXT'])

Admission Date:  [**2125-2-9**]              Discharge Date:   [**2125-2-16**]


Service: MEDICINE

Allergies:
Zocor / Lescol

Attending:[**Doctor Last Name 1857**]
Chief Complaint:
Chest pain

Major Surgical or Invasive Procedure:
Central venous line insertion (right internal jugular vein)

History of Present Illness:
Mr. [**Known lastname 1858**] is an 84 yo man with moderate aortic stenosis (outside
hospital echo in [**2124**] with [**Location (un) 109**] 1 cm2, gradient 28 mmHg, moderate
mitral regurgitation, mild aortic insufficiency), chronic left
ventricular systolic heart failure with EF 25-30%, hypertension,
hyperlipidemia, diabetes mellitus, CAD s/p CABG in [**2099**] with
SVG-LAD-Diagonal, SVG-OM, and SVG-RPDA-RPL, with a re-do CABG in
[**9-/2117**] with LIMA-LAD, SVG-OM, SVG-diagonal, and SVG-RCA. He also
has severe peripheral arterial disease s/p peripheral bypass
surgery. He presented to [**Hospital 1474**] Hospital ER this morning with
shortness of breath and chest pain 

In [ ]:
# Sort by subject_ID, HAD_ID then CHARTDATE
df_notes = df_notes.sort_values(by=['SUBJECT_ID','HADM_ID','CHARTDATE'])
# Merge notes table to admissions table
df_adm_notes = pd.merge(df_adm[['SUBJECT_ID','HADM_ID','ADMITTIME','DISCHTIME','DAYS_NEXT_ADMIT','NEXT_ADMITTIME','ADMISSION_TYPE','DEATHTIME','OUTPUT_LABEL','DURATION']],
                        df_notes[['SUBJECT_ID','HADM_ID','CHARTDATE','TEXT','CATEGORY', 'CHARTTIME']],
                        on = ['SUBJECT_ID','HADM_ID'],
                        how = 'left')

In [ ]:
# Grab date only, not the time
df_adm_notes['ADMITTIME_C'] = df_adm_notes.ADMITTIME.apply(lambda x: str(x).split(' ')[0])
df_adm_notes['ADMITTIME_C'] = pd.to_datetime(df_adm_notes.ADMITTIME_C, format = '%Y-%m-%d', errors = 'coerce')

#make sure that chartdate is in appropriate format
df_adm_notes['CHARTDATE'] = pd.to_datetime(df_adm_notes.CHARTDATE, format = '%Y-%m-%d', errors = 'coerce')

# make sure that charttime is in appropriate format
df_adm_notes['CHARTTIME'] = pd.to_datetime(df_adm_notes.CHARTTIME, errors = 'coerce')

In [ ]:
# Gather Discharge Summaries Only
df_discharge = df_adm_notes[df_adm_notes['CATEGORY'] == 'Discharge summary']
# multiple discharge summary for one admission -> after examination -> replicated summary -> replace with the last one
df_discharge = (df_discharge.groupby(['SUBJECT_ID','HADM_ID']).nth(-1)).reset_index()
df_discharge=df_discharge[df_discharge['TEXT'].notnull()]

In [ ]:
#Handle missing data in Echo, ECG and Discharge note categories
def fill_missing_charttime(row):
  '''this function fills in chart time for the categories missing data (echo, ECG and discharge)
  '''
  if pd.isna(row['CHARTTIME']):
    return max(row['CHARTDATE'], row['ADMITTIME'])
  else:
    return row['CHARTTIME']

df_adm_notes['filled_charttime'] = df_adm_notes.apply(lambda x: fill_missing_charttime(x), axis=1)

In [ ]:
def less_n_days_data(df_adm_notes, n):
  '''Function pulls all notes for a given admission that were entered up to n DAYS after the intitial day of admission.
  It then concatenates all notes for a given admission into one long TEXT feature.

  Note: this groups notes by calendar day and does not represent a specific timeframe in hours
  '''
  df_less_n = df_adm_notes[
      ((df_adm_notes['CHARTDATE'] - df_adm_notes['ADMITTIME_C']).dt.total_seconds() / (24 * 60 * 60)) < n]
  df_less_n = df_less_n[df_less_n['TEXT'].notnull()]
  # concatenate notes into one
  df_concat = pd.DataFrame(df_less_n.groupby('HADM_ID')['TEXT'].apply(lambda x: "%s" % ' '.join(x))).reset_index()
  df_concat['OUTPUT_LABEL'] = df_concat['HADM_ID'].apply(
      lambda x: df_less_n[df_less_n['HADM_ID'] == x].OUTPUT_LABEL.values[0])
  
  return df_concat

def less_n_hours_data(df_adm_notes, n):
  '''Function pulls all notes for a given admission that were entered up to n HOURS after the intitial day of admission.
  It then concatenates all notes for a given admission into one long TEXT feature.
  
  Note: this groups notes by calendar hour - this is a departure from the initial work published by Huang and Imasogie
  '''
  df_less_n = df_adm_notes[
      ((df_adm_notes['filled_charttime'] - df_adm_notes['ADMITTIME']).dt.total_seconds() / (60 * 60)) < n]
  df_less_n = df_less_n[df_less_n['TEXT'].notnull()]
  # concatenate notes into one
  df_concat = pd.DataFrame(df_less_n.groupby('HADM_ID')['TEXT'].apply(lambda x: "%s" % ' '.join(x))).reset_index()
  df_concat['OUTPUT_LABEL'] = df_concat['HADM_ID'].apply(
      lambda x: df_less_n[df_less_n['HADM_ID'] == x].OUTPUT_LABEL.values[0])
  
  return df_concat

In [ ]:
#this took about 10 min to run

##subset data by day cutoffs - this is what's done in the original paper.
# df_less_2_days = less_n_days_data(df_adm_notes, 2)
# df_less_3_days = less_n_days_data(df_adm_notes, 3)

##subset data by hour cutoffs - this is a new iteration I created for comparison
df_less_24_hours = less_n_hours_data(df_adm_notes, 24)
# df_less_48_hours = less_n_hours_data(df_adm_notes, 48)
# df_less_72_hours = less_n_hours_data(df_adm_notes, 72)

In [ ]:
import re

def preprocess1(x):
    y = re.sub('\\[(.*?)\\]', '', x)  # remove de-identified brackets
    y = re.sub('[0-9]+\.', '', y)  # remove 1.2. since the segmenter segments based on this
    y = re.sub('dr\.', 'doctor', y)
    y = re.sub('m\.d\.', 'md', y)
    y = re.sub('admission date:', '', y)
    y = re.sub('discharge date:', '', y)
    y = re.sub('--|__|==', '', y)
    return y

In [ ]:
from tqdm import tqdm, trange

In [ ]:
def preprocessing(df_less_n):
    '''
    step 1: clean text, remove new lines and returns strip and lower caps
    step 2: apply regex function preprocess1
    step 3: break up long text blocks into 318 word chunks - NOTE this is something we can tune later if we want to
    '''
    df_less_n['TEXT'] = df_less_n['TEXT'].fillna(' ')
    df_less_n['TEXT'] = df_less_n['TEXT'].str.replace('\n', ' ')
    df_less_n['TEXT'] = df_less_n['TEXT'].str.replace('\r', ' ')
    df_less_n['TEXT'] = df_less_n['TEXT'].apply(str.strip)
    df_less_n['TEXT'] = df_less_n['TEXT'].str.lower()

    df_less_n['TEXT'] = df_less_n['TEXT'].apply(lambda x: preprocess1(x))

    # to get 318 words chunks for readmission tasks
    df_len = len(df_less_n)
    want = pd.DataFrame({'ID': [], 'TEXT': [], 'Label': []})
    for i in tqdm(range(df_len)):
        x = df_less_n.TEXT.iloc[i].split()
        n = int(len(x) / 318)
        for j in range(n):
            want = want.append({'TEXT': ' '.join(x[j * 318:(j + 1) * 318]), 'Label': df_less_n.OUTPUT_LABEL.iloc[i],
                                'ID': df_less_n.HADM_ID.iloc[i]}, ignore_index=True)
        if len(x) % 318 > 10:
            want = want.append({'TEXT': ' '.join(x[-(len(x) % 318):]), 'Label': df_less_n.OUTPUT_LABEL.iloc[i],
                                'ID': df_less_n.HADM_ID.iloc[i]}, ignore_index=True)

    return want

The preprocessing below for the Discharge, 2-Day and 3-Day lookback windows took a while.  Times below:

*   1-day (by hour): 50 min
*   2-day (by hour): 1hr 20 min
*   3-day (by hour): 2hr 20 min


Note: Times seem to be reduced when using an accelerator - consuder activating the GPU before running this cell 

Uncomment the lines below (I've commented it out since I've already run preprocessing and pickled the files).

In [ ]:
#uncomment individual lines below to run the preprocessing on each cutoff dataframe
df_less_1_by_hour = preprocessing(df_less_24_hours)
# df_less_2_by_hour = preprocessing(df_less_48_hours)
# df_less_3_by_hour = preprocessing(df_less_72_hours)

# df_less_2 = preprocessing(df_less_2)
# df_less_3 = preprocessing(df_less_3)

# df_discharge = preprocessing(df_discharge)

100%|██████████| 43921/43921 [45:29<00:00, 16.09it/s]


In [ ]:
df_less_1_by_hour.head()
len(df_less_1_by_hour)

238835

Run the code below to save preprocessed data out to pickle files

In [ ]:
# df_less_1_by_hour.to_pickle("drive/MyDrive/266: NLP/w266: NLP Project/pickle_files/df_less_1_by_hour.pkl")
# df_less_2_by_hour.to_pickle("drive/MyDrive/266: NLP/w266: NLP Project/pickle_files/df_less_2_by_hour.pkl")
# df_less_3_by_hour.to_pickle("drive/MyDrive/266: NLP/w266: NLP Project/pickle_files/df_less_3_by_hour.pkl")

Run code below to read pickle files back in

In [ ]:
# df_discharge = pd.read_pickle('drive/MyDrive/266: NLP/w266: NLP Project/pickle_files/df_discharge.pkl')

# df_less_2 = pd.read_pickle('drive/MyDrive/266: NLP/w266: NLP Project/pickle_files/df_less_2.pkl')
# df_less_3 = pd.read_pickle('drive/MyDrive/266: NLP/w266: NLP Project/pickle_files/df_less_3.pkl')

# df_less_1_by_hour = pd.read_pickle('drive/MyDrive/266: NLP/w266: NLP Project/pickle_files/df_less_1_by_hour.pkl')
# df_less_2_by_hour = pd.read_pickle('drive/MyDrive/266: NLP/w266: NLP Project/pickle_files/df_less_2_by_hour.pkl')
df_less_3_by_hour = pd.read_pickle('/content/drive/MyDrive/w266: NLP Project/pickle_files/df_less_3_by_hour.pkl')

In [ ]:
#check picke file shapes
# print('dischage df: {}'.format(df_discharge.shape))
# print('2-day df: {}'.format(df_less_2.shape))
# print('3-day df: {}'.format(df_less_3.shape))
# print('1-day df (parsed by hour): {}'.format(df_less_1_by_hour.shape))
# print('2-day df (parsed by hour): {}'.format(df_less_2_by_hour.shape))
print('3-day df (parsed by hour): {}'.format(df_less_3_by_hour.shape))

3-day df (parsed by hour): (454798, 3)


# Train-Test split
An example to get the train/test/split with random state:
note that we divide on patient admission level and share among experiments, instead of notes level. 
This way, since our methods run on the same set of admissions, we can see the progression of readmission scores. 

In [ ]:
#pull list of admissions ID where the patient re-admitted/ did not readmit
readmit_ID = df_adm[df_adm.OUTPUT_LABEL == 1].HADM_ID
not_readmit_ID = df_adm[df_adm.OUTPUT_LABEL == 0].HADM_ID

# subsampling to get the balanced pos/neg numbers of patients for each dataset
not_readmit_ID_use = not_readmit_ID.sample(n=len(readmit_ID), random_state=1)

#list of IDs for the validation and test sets - currenlty 20% in this group
id_val_test_t = readmit_ID.sample(frac=0.2, random_state=1)
id_val_test_f = not_readmit_ID_use.sample(frac=0.2, random_state=1)

#list of IDs for training set - excluding all IDs in validation set - currently 80% of total data
id_train_t = readmit_ID.drop(id_val_test_t.index)
id_train_f = not_readmit_ID_use.drop(id_val_test_f.index)

#subset the va/test group created above - half go to final val set and half to final test set
id_val_t = id_val_test_t.sample(frac=0.5, random_state=1)
id_val_f = id_val_test_f.sample(frac=0.5, random_state=1)
id_test_t = id_val_test_t.drop(id_val_t.index)
id_test_f = id_val_test_f.drop(id_val_f.index)

# test if there is overlap between train and test, should return "array([], dtype=int64)"
print('Overlapping IDs: {}'.format((pd.Index(id_test_t).intersection(pd.Index(id_train_t))).values))

#concat the test IDs and add appropriate flag
id_test = pd.concat([id_test_t, id_test_f])
test_id_label = pd.DataFrame(data=list(zip(id_test, [1] * len(id_test_t) + [0] * len(id_test_f))),
                             columns=['id', 'label'])

#concat the validation IDs and add appropriate flag
id_val = pd.concat([id_val_t, id_val_f])
val_id_label = pd.DataFrame(data=list(zip(id_val, [1] * len(id_val_t) + [0] * len(id_val_f))), columns=['id', 'label'])

#concat the training IDs and add appropriate flag
id_train = pd.concat([id_train_t, id_train_f])
train_id_label = pd.DataFrame(data=list(zip(id_train, [1] * len(id_train_t) + [0] * len(id_train_f))),
                              columns=['id', 'label'])

Overlapping IDs: []


# Save data for 24 hours after admission

### Output final .csv files to drive

**Note:** there is some concern about the length of notes for each admission.  There seems to be longer notes for more complex admissions, which inherantly have a higher risk of readmission.  The original authors try to mitigate this issue by adding some negative examples to the training data. 

In [ ]:
#subset the data to only include appropriate examples for each train/val/test subset
early_train = df_less_3_by_hour[df_less_3_by_hour.ID.isin(train_id_label.id)]
early_val = df_less_3_by_hour[df_less_3_by_hour.ID.isin(val_id_label.id)]

# we want to test on admissions that are not discharged already. So for less than 3 days of notes experiment,
# we filter out admissions discharged within 3 days
actionable_ID_1days = df_adm[df_adm['DURATION'] >= 1].HADM_ID
test_actionable_id_label = test_id_label[test_id_label.id.isin(actionable_ID_1days)]
early_test = df_less_3_by_hour[df_less_3_by_hour.ID.isin(test_actionable_id_label.id)]

# early_test = df_less_1_by_hour[df_less_1_by_hour.ID.isin(test_id_label.id)]

#run the cell below to ensure a reasonably balanced dataset
print('Training: {:0.2%} positive examples | {:,} total examples'.format(early_train['Label'].mean(), 
                                                                       early_train.shape[0]))
print('Validation: {:0.2%} positive examples | {:,} total examples'.format(early_val['Label'].mean(), 
                                                                       early_val.shape[0]))
print('Test: {:0.2%} positive examples | {:,} total examples'.format(early_test['Label'].mean(), 
                                                                       early_test.shape[0]))
early_train.to_csv('/content/drive/MyDrive/w266: NLP Project/data/3days_by_hour/train.csv', index=False)
early_val.to_csv('/content/drive/MyDrive/w266: NLP Project/data/3days_by_hour/val.csv', index=False)
early_test.to_csv('/content/drive/MyDrive/w266: NLP Project/data/3days_by_hour/test.csv', index=False)


Training: 55.06% positive examples | 52,363 total examples
Validation: 52.30% positive examples | 6,981 total examples
Test: 53.47% positive examples | 7,256 total examples


## Manually create unbalanced test set

In [ ]:
df_original_train = pd.read_csv('/content/drive/MyDrive/w266: NLP Project/data/3days_by_hour/train.csv')
original_train_ids = set(df_original_train['ID'])

In [ ]:
#pull list of admissions ID where the patient re-admitted/ did not readmit
readmit = df_adm[df_adm.OUTPUT_LABEL == 1]
not_readmit = df_adm[df_adm.OUTPUT_LABEL == 0]

# subsampling to get the balanced pos/neg numbers of patients for each dataset
readmit_ID_use = readmit[~readmit['HADM_ID'].isin(original_train_ids)]['HADM_ID']
not_readmit_ID_use = not_readmit[~not_readmit['HADM_ID'].isin(original_train_ids)]['HADM_ID']

#we have 618 positive examples, to ensure we retain the natural 6.5% + examples we see in the unbalanced
# data, we need a total of 9507 examples with 8889 negative exampless
unbalanced_not_readmit_ids = not_readmit_ID_use.sample(8889, random_state=42)
unbalanced_readmit_ids = readmit_ID_use

unbalanced_test_ids = unbalanced_not_readmit_ids.append(unbalanced_readmit_ids)
print('Test set size: {}'.format(unbalanced_test_ids.shape[0]))
print('Positive examples in test set: {}'.format(unbalanced_readmit_ids.shape[0]))

# #use IDs to create df of pre-processed & chunked training data
unbalanced_test_df = df_less_3_by_hour[df_less_3_by_hour['ID'].isin(unbalanced_test_ids)]

#check label balance in the final sequence breakout
# unbalanced_test_df.groupby('Label').agg({'Label':'count'})

#save df
unbalanced_test_df.to_csv(
    '/content/drive/MyDrive/w266: NLP Project/data/3days_by_hour/unbalanced_test.csv', index=False)

Test set size: 9507
Positive examples in test set: 618


In [3]:
df = pd.read_csv('/content/drive/MyDrive/w266: NLP Project/data/3days_by_hour/unbalanced_test.csv')

print('Test: {:0.2%} positive examples | {:,} total examples'.format(df['Label'].mean(), 
                                                                       df.shape[0]))

Test: 7.91% positive examples | 95,542 total examples


In [ ]:
# df_original_train[df_original_train['ID'] == 167791]
# df_original_train.head()
len(unbalanced_not_readmit_ids)

9507